In [7]:
from pydantic import BaseModel
import json
import datetime
from typing import Optional

class Video(BaseModel):
    url: str
    title: str
    duration: int
    date: str
    description: Optional[str] = None

whisper_model_speedup = 43 # 43x faster than real time

with open('./le_blob_videos.json') as f:
    data = [Video(**item) for item in json.load(f)]
    total_videos_duration_seconds = sum(video.duration for video in data)
    whisper_time = total_videos_duration_seconds/whisper_model_speedup
    total_videos_duration_formatted = str(datetime.timedelta(seconds=int(total_videos_duration_seconds)))
    whisper_time_formatted = str(datetime.timedelta(seconds=int(whisper_time)))
    print(f"total video time : {total_videos_duration_formatted}\nwhisper estimated time : {whisper_time_formatted}" )


total video time : 20 days, 17:36:10
whisper estimated time : 11:34:19


In [20]:
from transformers import pipeline

# Utiliser un pipeline de classification zéro-shot
classifier = pipeline("zero-shot-classification", model="mtheo/camembert-base-xnli")
# classifier = pipeline("zero-shot-classification", model="cmarkea/distilcamembert-base-nli")

# Fonction pour diviser le document en segments
def split_into_segments(text, max_length=512):
    words = text.split()
    segments = []
    current_segment = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1
        if current_length > max_length:
            segments.append(" ".join(current_segment))
            current_segment = []
            current_length = len(word) + 1
        current_segment.append(word)

    if current_segment:
        segments.append(" ".join(current_segment))

    return segments

# Documents à classifier
documents = [video.description for video in data[:100]]

# Catégories à prédire avec descriptions
# descriptions done using gpt : ```fait moi un paragraphe concis sur le theme {chapter title} qui traite tous les mots cles du sujet.```
categories = [
    {"label": "paleoclimats", "description": "La paléoclimatologie est la science qui étudie les climats passés et leurs variations. Elle tente d'établir les conditions environnementales caractéristiques de chaque période géoclimatique, notamment en termes de paléotempératures de l'atmosphère, des océans et des continents. Ces reconstitutions des variations climatiques passées, et éventuellement de leurs causes, apportent des données (en partie empiriques) sur l'évolution du climat actuel et futur."},
    {"label": "cellule", "description": "'La cellule, unité structurale et fonctionnelle du vivant,' souligne l'importance fondamentale des cellules dans la constitution et le fonctionnement des organismes. Chaque cellule est une entité autonome capable de réaliser des processus vitaux tels que la respiration, la reproduction, et la réponse aux stimuli. Les cellules possèdent des organites spécialisés, comme le noyau qui abrite l'ADN, les mitochondries pour la production d'énergie, et le réticulum endoplasmique pour la synthèse des protéines. En tant qu'unités de base, les cellules assurent la croissance, le développement et la maintenance des tissus et des organes, illustrant ainsi leur rôle central dans la vie et la complexité biologique."},
    {"label": "organisme", "description": "'L’organisme, une société de cellules' met en lumière la complexité et l'organisation des êtres vivants, où chaque cellule joue un rôle crucial. Les cellules, unités de base du vivant, se spécialisent pour former différents tissus et organes, collaborant de manière harmonieuse pour maintenir les fonctions vitales. Cette organisation cellulaire repose sur des mécanismes de communication sophistiqués, tels que les signaux chimiques et électriques, assurant la coordination des activités biologiques. Les cellules souches illustrent cette flexibilité en ayant la capacité de se différencier en divers types cellulaires, soulignant l'interconnexion et la coopération essentielles à la vie."},
    {"label": "phylogenie", "description": "'Plans d’organisation du vivant et phylogénie' explore la diversité et l'évolution des organismes en retraçant leurs relations ancestrales. Les plans d'organisation, ou plans d'organisation corporelle, définissent les structures et les systèmes fondamentaux des organismes, comme la symétrie corporelle, les cavités corporelles, et les segments. La phylogénie, quant à elle, étudie les liens évolutifs entre les espèces, révélant leur histoire commune et les processus d'évolution qui ont conduit à la biodiversité actuelle. Grâce à des méthodes comme l'analyse de l'ADN et les fossiles, les scientifiques peuvent reconstruire l'arbre de la vie, montrant comment les différentes formes de vie sont interconnectées et ont évolué au fil du temps."}
]

# Conversion des catégories en listes de labels et descriptions
labels = [category["label"] for category in categories]
descriptions = [category["description"] for category in categories]

# Classification des documents
for doc in documents:
    segments = split_into_segments(doc, max_length=512)
    aggregated_results = {label: 0 for label in labels}

    for segment in segments:
        results = classifier(segment, candidate_labels=descriptions)
        for description, score in zip(results['labels'], results['scores']):
            label = [category["label"] for category in categories if category["description"] == description][0]
            aggregated_results[label] += score

    # Normaliser les scores
    total_score = sum(aggregated_results.values())
    if total_score == 0:
        continue
    normalized_results = {label: score / total_score for label, score in aggregated_results.items()}

    max_label = max(normalized_results, key=normalized_results.get)
    if (normalized_results[max_label] > 0.7):
        print("MAXLABEL", max_label, normalized_results[max_label])
        print(f"Document: {doc}...")  # Affiche une partie du document pour un aperçu
        print(f"Predicted categories: {normalized_results}")
        print("\n\n==========================\n\n")


MAXLABEL phylogenie 0.7460696462501042
Document: Plongée étrange et fascinante au coeur de la pourriture avec une série documentaire de 21 épisodes de 3 minutes, racontée par Denis Lavant,  à partir de données scientifiques les plus récentes. 
Montage : Olivier Esmein
 
Série "La Nuit du Vivant : voyage au coeur de la pourriture"....
Predicted categories: {'paleoclimats': 0.016409524579807443, 'cellule': 0.1769946601276494, 'organisme': 0.060526169042439, 'phylogenie': 0.7460696462501042}




MAXLABEL phylogenie 0.7155675172058723
Document: La lumière naturelle a un impact direct sur notre horloge biologique et sur notre bien être. Explications.

Une infographie de l’émission « Futuremag » diffusée sur Arte....
Predicted categories: {'paleoclimats': 0.026068159873386027, 'cellule': 0.1806187301397909, 'organisme': 0.07774559278095079, 'phylogenie': 0.7155675172058723}




